In [38]:
import pandas as pd
import numpy as np
import random as r
import matplotlib.pyplot as plt
from scipy import stats
from iminuit import Minuit                            
from numpy.linalg import inv
import sys
import inspect

In [4]:
sys.path.append('External_Functions')
from ExternalFunctions import *

In [39]:
def get_weighted_mean(x, x_err):
    mean_weighted = np.nansum(x/x_err**2) / np.nansum(1/x_err**2)
    err_weighted = np.sqrt(1/np.sum(1 / x_err**2)) 
    chi2 = np.nansum((x - mean_weighted)**2/x_err**2)
    p = stats.chi2.sf(chi2, len(x) - 1)
    return mean_weighted, err_weighted, chi2, p

def get_separation(x, y, ddof = 0):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    std_x = np.std(x, ddof = ddof)
    std_y = np.std(y, ddof = ddof)
    d = np.abs((mean_x - mean_y)) / np.sqrt(std_x**2 + std_y**2)
    return d


## functions

In [87]:
def gauss_pdf_norm(x, mu, sigma) :
    return 1.0 / np.sqrt(2*np.pi) / sigma * np.exp( -0.5 * (x-mu)**2 / sigma**2)

def scale_binned(func, N_data, N_bins, range_bins):
    def wrapper(*x):
        return func(*x) * N_data * range_bins / N_bins
    return wrapper



In [48]:
test_data = np.random.normal(loc = 5, scale = 8, size = 1000)



In [52]:
dataframe = pd.read_csv('DataAndCodeForProblemSet/data_CountryScores.csv', header=0, index_col=None)
Country, GDP, PopSize, HappinessI, EconomicFreedomI, PressFreedomI, EducationI = dataframe.values.T

In [53]:
GDP = GDP.astype('float64')

In [54]:
log_GDP = np.log(GDP)

In [105]:
N_bins = 50
bins_range = np.max(log_GDP) - np.min(log_GDP)
scale = len(GDP) * bins_range / N_bins

y, edges = np.histogram(log_GDP, bins = N_bins, range = (min(log_GDP), max(log_GDP)))
x = ((edges[1:] + edges[:-1])/2)

func = lambda x, mu, sigma: scale * gauss_pdf_norm(x, mu, sigma)

chi2_gauss = Chi2Regression(f = func, x = x, y = y, sy = np.sqrt(y))
fit = Minuit(chi2_gauss, mu = 7, sigma = 1) 
fit.errordef = 1
fit.migrad()         


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = inf                        │              Nfcn = 210              │
│ EDM = nan (Goal: 0.0002)         │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name  │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼───────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ mu    │     7     │    nan    │            │            │         │         │       │
│ 1 │ sigma │     1     │    nan    │            │            │         │         │       │
└───┴───────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───────┬─────────────┐
│       │    mu sigma │
├───────┼─────────────┤
│    mu │   nan   nan │
│ sigma │   nan   nan │
└───────┴─────────────┘